# Monte Carlo

This notebook creates two files. For a given number of market situations and a given set of prices, it creates features and saves them in the `demand_prediction_data` file. Next, for each market situation and each price, the notebook runs multiple simulations and tracks purchases. The notebook then calculates the probabilities of a purchase for each price and market situation and stores them in the `PEW_comparison` file. The time horizon is 1.

Because of performance reasons the number of simulations and the number of considered market situations used in the start configuration are smaller than the ones used for the data for the paper. The inline comments show the values used in the paper, which lead to a longer runtime.

In [ ]:
import numpy as np
import random
import csv
import matplotlib.pyplot as plt
import pandas as pd

Set start values.

In [ ]:
num_firm = 5                                   # the number of firms which we simulate
time_horizon = 1                               # set the time horizon which we watch to 1 
num_market_events = 2                          # the max price adjustments each firm will make
mean_betw_customer_arrival = 2                 # chose mean for how often customers appear
prices = np.arange(0,15.1,0.2)                 # values used for paper: (np.arange(0,15.1,0.1))
watched_firm = 0                            
num_sims = 100                                 # the number of simulations, value used for paper: 10000 
num_market_situations = 10                     # the number of market situations with different start scenarios, 
                                               # value used for paper: 40
num_customer_events = 4                        # the maximum number of customers
reaction_time_firm = time_horizon              # watched firm doesn't update price in time horizon
setups = ["S1","S2","S3"]                      # set the name of the setups from the paper
random.seed(12530586)                          # set a random seed to always get comparable results

In [ ]:
def explanatory_vars(explanatory_var, num_price, market_situation):
    return {
        '0' : 1,
        '1' : price_ranks_firm[market_situation][num_price],
        '2' : 1 if price_ranks_firm[market_situation][num_price]==1 else 0,
        '3' : qual_rank_firm[market_situation],
        '4' : rating_ranks[market_situation][watched_firm],
        '5' : prices[num_price],
        '6' : prices[num_price] - min([prices_firm_pred[j][market_situation] for j in range(num_firm) if j!=watched_firm]) if num_firm > 1 else 0,
        '7' : qual_firm_pred[watched_firm][market_situation],
        '8' : rating_firm_pred[watched_firm][market_situation],
        '9' : 15 - (prices[num_price] +
                    0.5*qual_firm_pred[watched_firm][market_situation] +
                    0.25*(100-rating_firm_pred[watched_firm][market_situation])) 
        if((prices[num_price] +
            0.5*qual_firm_pred[watched_firm][market_situation] +
            0.25*(100-rating_firm_pred[watched_firm][market_situation])) 
           < min([
               prices_firm_pred[j][market_situation] +
               0.5*qual_firm_pred[j][market_situation] +
               0.25*(100-rating_firm_pred[j][market_situation]) 
               for j in range(num_firm) 
               if j!= watched_firm]))
        else 0,
    }[str(explanatory_var)]

 Create the features for each starting situation from each market situation and save them as a file.

In [ ]:
prices_firm_pred = np.round(np.random.uniform(6, 16, size=(num_firm, num_market_situations)), decimals = 2)
                            
qual_firm_pred = np.round(np.random.uniform(0.5, 5.5, size=(num_firm, num_market_situations)))
                          
rating_firm_pred = np.round(np.random.uniform(90, 100, size=(num_firm, num_market_situations)), decimals = 1)

price_ranks_firm = np.array(
    [[
              1 +  
              sum([
                  1 
                  for i in range(num_firm) 
                  if prices[j]> prices_firm_pred[i][w] and i!=watched_firm]) 
              +
              sum([
                  1 
                  for i in range(num_firm) 
                  if prices[j]== prices_firm_pred[i][w] and i!=watched_firm])/2
              for j in range(len(prices))]
        for w in range(num_market_situations)])
                            
qual_rank_firm = np.array(
    [
         1 +  
         sum([
             1 
             for i in range(num_firm) 
             if qual_firm_pred[watched_firm][w]> qual_firm_pred[i][w] and i!=watched_firm]) 
         +
         sum([
             1 
             for i in range(num_firm) 
             if qual_firm_pred[watched_firm][w]== qual_firm_pred[i][w] and i!=watched_firm])/2
         for w in range(num_market_situations)])
                         
rating_ranks = np.array(
    [[
              1 +  
              sum([
                  1 
                  for i in range(num_firm) 
                  if rating_firm_pred[j][w]> rating_firm_pred[i][w] and i!=j]) 
              +
              sum([
                  1 
                  for i in range(num_firm) 
                  if rating_firm_pred[j][w]== rating_firm_pred[i][w] and i!=j])/2
              for j in range(num_firm)]
        for w in range(num_market_situations)])

num_explanatory_vars = 10

value_explanatory_vars = [[[
            explanatory_vars(explanatory_var, num_price, w) 
            for explanatory_var in range(num_explanatory_vars)] 
        for num_price in range(len(prices))]
    for w in range(num_market_situations)]

with open('demand_prediction_data_S3.csv', 'w') as file:
    writer = csv.writer(file, delimiter='\t')
    for market_situation in range(num_market_situations):
        for num_price in range(len(prices)):
            line = list(value_explanatory_vars[market_situation][num_price])
            line.remove(1)
            line.insert(0, prices[num_price])
            line.insert(0, market_situation+1)
            writer.writerow(line)

In [ ]:
def price_if_smaller_else(price, price_threshold, min_price, prices_firm, c):
    return(
        price 
        if min([prices_firm[c] for firm in range(num_firm) if firm!=c]) < price_threshold 
        else round(
            min([
                    max(min_price, prices_firm[firm] - 0.5) for firm in range(num_firm) if firm != c])
            ,2))

s1 = {
        '0' : lambda prices_firm, c: round(random.uniform(0,15),2),
        '1' : lambda prices_firm, c: round(random.uniform(5,15),2),
        '2' : lambda prices_firm, c: round(random.uniform(5,15),2),
        '3' : lambda prices_firm, c: round(random.uniform(5,15),2),
        '4' : lambda prices_firm, c: round(random.uniform(5,15),2)
    }

s2 = {
        '0' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c),
        '1' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c),
        '2' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c),
        '3' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c),
        '4' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c)
    }

s3 = {
        '0' : lambda prices_firm, c: price_if_smaller_else(10,5,4.9,prices_firm,c),
        '1' : lambda prices_firm, c: price_if_smaller_else(9,4,3.9,prices_firm,c),
        '2' : lambda prices_firm, c: price_if_smaller_else(12,6,5.9,prices_firm,c),
        '3' : lambda prices_firm, c: 11,
        '4' : lambda prices_firm, c: 13
    }

In [ ]:
def calc_prices(prices_firm_adjust, adjusting_firm, setup):
    prices_firm_adjust = np.array(prices_firm_adjust)
    #weird indexing because there are actually no price adjustments at time 0 nor on the price horizon
    for i in range(1,len(prices_firm_adjust)-1):        
        prices_firm_adjust[i:,adjusting_firm[i]] = setup[str(adjusting_firm[i])](prices_firm_adjust[i-1],adjusting_firm[i])
    return prices_firm_adjust

Calculate all the necessary information for when firms adjusts their prices and how often they adjust their prices.  

In [ ]:
time_betw_market_events_firm = np.random.uniform(0.8, 1.2, size = (len(prices), num_market_situations, num_sims, num_firm, num_market_events))

time_adjusts_firm = time_betw_market_events_firm

time_adjusts_firm[:, :, :, :, 0] = np.random.uniform(0, 1, size = (len(prices), num_market_situations, num_sims, num_firm))

# calculate the time when each company adjusts their prices
for p in range(len(prices)):
    for w in range(num_market_situations):
        for sim_num in range(num_sims):
            for c in range(num_firm):
                for i in range(1,num_market_events):
                    time_adjusts_firm[p,w,sim_num,c,i] = time_adjusts_firm[p,w,sim_num,c,i-1] + time_betw_market_events_firm[p,w,sim_num,c,i]  

price_adjusts_before_horizon = np.array(
    [[[
                set([
                    time_adjusts_firm[p,w,sim_num].flatten()[i] 
                    for i in range(len(time_adjusts_firm[p,w,sim_num].flatten())) 
                    if time_adjusts_firm[p,w,sim_num].flatten()[i] < time_horizon]) 
                for sim_num in range(num_sims)]
            for w in range(num_market_situations)]
        for p in range(len(prices))])

price_adjusts_before_horizon = np.array(
    [[[
                [0] + sorted(price_adjusts_before_horizon[p,w,sim_num]) + [time_horizon] 
                for sim_num in range(num_sims)]
            for w in range(num_market_situations)]
        for p in range(len(prices))])

num_price_adjusts_before_horizon = np.array(
    [[[
                len(price_adjusts_before_horizon[p,w,sim_num]) 
                for sim_num in range(num_sims)]
            for w in range(num_market_situations)]
        for p in range(len(prices))])

adjusting_firm = np.array(
    [[[[
                    min(np.where(time_adjusts_firm[p,w,sim_num]==price_adjusts_before_horizon[p,w,sim_num][i] )[0]) 
                    if price_adjusts_before_horizon[p,w,sim_num][i]!= time_horizon and i!=0 
                    else -1 
                    for i in range(num_price_adjusts_before_horizon[p,w,sim_num])] 
                for sim_num in range(num_sims)]
            for w in range(num_market_situations)]
        for p in range(len(prices))])

Calculate when customers will arrive and how they will score the quality and rating of all firms.

In [ ]:
time_betw_customer_events = mean_betw_customer_arrival * np.random.exponential(size = (len(prices), num_market_situations, num_sims, num_customer_events))

time_customer_events = time_betw_customer_events

#calculate the arrival of customers after the first one
for p in range(len(prices)):
    for w in range(num_market_situations):
        for sim_num in range(num_sims):
            for c in range(1, num_customer_events):
                time_customer_events[p,w,sim_num,c] = time_customer_events[p,w,sim_num,c-1] + time_betw_customer_events[p,w,sim_num,c]

rand_customer_score_qual = np.random.uniform(0, 1, size = (len(prices), num_market_situations ,num_sims, num_customer_events))

rand_customer_score_rating = np.random.uniform(0, 0.5, size = (len(prices), num_market_situations ,num_sims, num_customer_events))

#calculate the score from each customer for each firm based on quality and rating of the offers
customer_score_qual_rating = np.array(
    [[[[[
                        (rand_customer_score_qual[p,w,s][c]*qual_firm_pred[k,w] +
                         rand_customer_score_rating[p,w,s][c]*(100-rating_firm_pred[k,w]))
                        for k in range(num_firm)] 
                    for c in range(num_customer_events)] 
                for s in range(num_sims)]
            for w in range(num_market_situations)]
        for p in range(len(prices))])

Calculate the prices adjustments for each firm in the time horizon.

In [ ]:
prices_firm = np.array(
    [[[
                prices_firm_pred[k,w] 
                if k!=watched_firm 
                else prices[a] 
                for w in range(num_market_situations)] 
            for a in range(len(prices))]  
        for k in range(num_firm)])

prices_firm_adjust = np.array(
    [[[[[
                        prices_firm[k,a,w]  
                        for k in range(num_firm)]
                    for _ in range(num_price_adjusts_before_horizon[a,w,s])] 
                for s in range(num_sims)] 
            for w in range(num_market_situations)] 
        for a in range(len(prices))])

prices_firm_adjust = np.array(
    [[[[
                    np.array(calc_prices(prices_firm_adjust[a,w,s], adjusting_firm[a,w,s], globals()["s{}".format(setup+1)])) 
                    for s in range(num_sims)] 
                for w in range(num_market_situations)] 
            for a in range(len(prices))]
        for setup in range(len(setups))])

As we don't want to know what happens after or on the time horizon, we will always substract 1 when using the value num_price_adjusts_before_horizon in the following code.

In [ ]:
# get all the customers that appear between market events, so we don't always have to iterate through each customer
gen = np.array(
    [[[[[
                        x 
                        for x in range(num_customer_events) 
                        if price_adjusts_before_horizon[a,w,s][i]< time_customer_events[a,w,s][x]< price_adjusts_before_horizon[a,w,s][i+1]] 
                    for i in range(num_price_adjusts_before_horizon[a,w,s] - 1)] 
                for s in range(num_sims)] 
            for w in range(num_market_situations)] 
        for a in range(len(prices))])

Simulate the customer by completing the score using the current prices of each firm and then calculating a decision for each customer.

In [ ]:
rand_customer_score = np.array(
    [[[[[[[
                                (prices_firm_adjust[setup,a,w,s][i,k] + customer_score_qual_rating[a,w,s,c,k]) 
                                for k in range(num_firm)]
                            for c in gen[a,w,s][i]]
                        for i in range(num_price_adjusts_before_horizon[a,w,s] - 1)] 
                    for s in range(num_sims)] 
                for w in range(num_market_situations)] 
            for a in range(len(prices))]
        for setup in range(len(setups))])

rand_customer_decision = np.array(
    [[[[[[
                            np.argmin(rand_customer_score[setup,a,w,s][i][c]) 
                            if min(rand_customer_score[setup,a,w,s][i][c])< random.uniform(5,15) 
                            else -1 
                            for c in range(len(gen[a,w,s][i]))]
                        for i in range(num_price_adjusts_before_horizon[a,w,s] - 1)] 
                    for s in range(num_sims)] 
                for w in range(num_market_situations)] 
            for a in range(len(prices))]
        for setup in range(len(setups))])

Calculate different metrics and save them.

In [ ]:
# count how often a firm sold anything in each simulation
As = np.array(
    [[[[[
                        np.count_nonzero(np.hstack(rand_customer_decision[setup,a,w,s])== k) 
                        for k in range(num_firm)]
                    for s in range(num_sims)] 
                for w in range(num_market_situations)] 
            for a in range(len(prices))]
        for setup in range(len(setups))])

# count how often a firm sold a certain something a certain number of times in all the simulations 
A = np.array(
    [[[[[
                        1/num_sims*np.count_nonzero(As[setup,a,w,:,k]==x) 
                        for k in range(num_firm)]
                    for x in range(6)]
                for w in range(num_market_situations)] 
            for a in range(len(prices))]
        for setup in range(len(setups))])

# probability that the watched firm sold something
P_ori = [[[
            1- A[setup,a,w,0,watched_firm]
            for w in range(num_market_situations)] 
        for a in range(len(prices))]
    for setup in range(len(setups))]

# probability that the watched firm sold more than 2 articles
P_ori2 = [[[
            sum([
                A[setup,a,w,x,watched_firm]
                for x in range(2,6)])
            for w in range(num_market_situations)] 
        for a in range(len(prices))]
    for setup in range(len(setups))]

# the estimated profit
EW_ori = [[[
            prices[a]*P_ori[setup][a][w] 
            for w in range(num_market_situations)]
        for a in range(len(prices))]
    for setup in range(len(setups))]

for setup in range(len(setups)):
    with open('PEW_comparison_{}.csv'.format(setups[setup]), 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        for w in range(num_market_situations):
            for a in range(len(prices)):
                line = [w+1,prices[a],P_ori[setup][a][w], P_ori2[setup][a][w], EW_ori[setup][a][w]]
                writer.writerow(line)